# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
#api_url = f'https://cloud.iexapis.com/stable/tops?token={IEX_CLOUD_API_TOKEN}&symbols={symbol}' #free
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}' #paid

data = requests.get(api_url).json()
print(data)


{'avgTotalVolume': 65443378, 'calculationPrice': 'close', 'change': -2.15, 'changePercent': -0.01361, 'close': 155.81, 'closeSource': 'official', 'closeTime': 1662148800416, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 155.615, 'delayedPriceTime': 1662148781927, 'extendedChange': -0.16, 'extendedChangePercent': -0.00103, 'extendedPrice': 155.65, 'extendedPriceTime': 1662163197667, 'high': 160.362, 'highSource': '15 minute delayed price', 'highTime': 1662148799986, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 155.845, 'iexCloseTime': 1662148799399, 'iexLastUpdated': 1662149227300, 'iexMarketPercent': 0.018617029017785443, 'iexOpen': 159.87, 'iexOpenTime': 1662125400491, 'iexRealtimePrice': 156.11, 'iexRealtimeSize': 1, 'iexVolume': 1432725, 'lastTradeTime': 1662148799986, 'latestPrice': 155.81, 'latestSource': 'Close', 'latestTime': 'September 2, 2022', 'latestUpdate': 1662148800416, 'latestVolume': 76957768, 'low': 154.965, 'lowSou

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['iexAskPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']

final_dataframe = pd.DataFrame(columns = my_columns)

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = my_columns
    ),
    ignore_index = True
)

C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\4129068022.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,0,2503983869120,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
"""
final_dataframe = pd.DataFrame(columns = my_columns)

for stock in stocks['Ticker']:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}' #paid
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(pd.Series(
        [
            stock, 
            data['iexAskPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns
        ),
        ignore_index = True
    )
"""

"\nfinal_dataframe = pd.DataFrame(columns = my_columns)\n\nfor stock in stocks['Ticker']:\n    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}' #paid\n    data = requests.get(api_url).json()\n    final_dataframe = final_dataframe.append(pd.Series(\n        [\n            stock, \n            data['iexAskPrice'],\n            data['marketCap'],\n            'N/A'\n        ],\n        index = my_columns\n        ),\n        ignore_index = True\n    )\n"

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [9]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [10]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

#GET /stock/{symbol}/batch
#/stock/market/batch?symbols=aapl,fb,tsla&types=quote,news,chart&range=1m&last=5
for symbol_string in symbol_strings:  #[:1]: #not include the first one. 
    batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    
    for symbol in symbol_string.split(','):
        try: 
            print(f"Starting {symbol}, {data[symbol]['quote']['latestPrice']}")
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [ 
                        symbol,
                        data[symbol]['quote']['latestPrice'],
                        data[symbol]['quote']['marketCap'],
                        'N/A'
                    ], index = my_columns 
                ), 
                ignore_index=True

            )
        except:
            pass

final_dataframe

    






Starting A, 128.01
Starting AAL, 12.99
Starting AAP, 171.73
Starting AAPL, 155.81
Starting ABBV, 136.28
Starting ABC, 147.71
Starting ABMD, 261.24
Starting ABT, 102.5
Starting ACN, 284.07

C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\L


Starting ADBE, 368.14
Starting ADI, 148.63
Starting ADM, 87.61
Starting ADP, 238.48
Starting ADSK, 198.69
Starting AEE, 92.93
Starting AEP, 101.09
Starting AES, 26.15
Starting AFL, 59.4

C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\L


Starting AIG, 52.41
Starting AIV, 8.76
Starting AIZ, 158.9
Starting AJG, 180.47
Starting AKAM, 88.26
Starting ALB, 259.88
Starting ALGN, 244.51
Starting ALK, 43.62
Starting ALL, 121.48
Starting ALLE, 94.89
Starting ALXN, 182.5

...

Starting WDC, 41.65
Starting WEC, 103.01
Starting WELL, 75.83
Starting WFC, 43.38
Starting WHR, 153.69
Starting WM, 168.45
Starting WMB, 33.72
Starting WMT, 133
Starting WRB, 64.77
Starting WRK, 39.64
Starting WST, 291.35
Starting WU, 14.56
Starting WY, 33.87
Starting WYNN, 59.18
Starting XEL, 74.41
Starting XLNX, 194.92
Starting XOM, 95.59
Starting XRAY, 31.8
Starting XRX, 15.79
Starting XYL, 91.21
Starting YUM, 112.74
Starting ZBH, 105.57


C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\L

Starting ZBRA, 297.6
Starting ZION, 55.04
Starting ZTS, 156.98


C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_5920\3174184726.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.01,38237618249,N/A
1,AAL,12.99,8441498592,N/A
2,AAP,171.73,10324105183,N/A
3,AAPL,155.81,2503983869120,N/A
4,ABBV,136.28,240956190339,N/A
...,...,...,...,...
496,YUM,112.74,32079318519,N/A
497,ZBH,105.57,22150649682,N/A
498,ZBRA,297.6,15412686442,N/A
499,ZION,55.04,8281944480,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [11]:
val =  100000 #input("Enter the value of your portfolio:")

In [12]:
position_size = val/len(final_dataframe.index)

for i in range (0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,128.01,38237618249,1
1,AAL,12.99,8441498592,15
2,AAP,171.73,10324105183,1
3,AAPL,155.81,2503983869120,1
4,ABBV,136.28,240956190339,1
...,...,...,...,...
496,YUM,112.74,32079318519,1
497,ZBH,105.57,22150649682,1
498,ZBRA,297.6,15412686442,0
499,ZION,55.04,8281944480,3


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [25]:
writer = pd.ExcelWriter('Recommended Trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [26]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [27]:
"""
writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
writer.save()
"""


"""
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capital', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)
"""


0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [28]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1]) #all entries
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format) #column title, first entry of above list



## Saving Our Excel Output

Saving our Excel file is very easy:

In [29]:
writer.save()